## Тема: Введение, примеры задач, бизнес- и ML-метрики 

In [1]:
import numpy as np
import pandas as pd

### 1. Сравните метрики hit_rate@k, precision@k.<br>Какую метрику использовать предпочтительно и почему?<br>Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

hit_rate@k - возвращает бинарное значение, о наличии релевантного(-ных) товара в списке рекомендованных.  
precision@k - возвращает долю релевантных товаров из списка рекомендованных.

precision@k - использовать предпочтительней, т.к. метрика является более информативной.

Использование метрики hit_rate@k более уместно в случаях требующих односложный ответ:
* Рекомендовано топ-10 товаров случайным людям. Была ли совершена хоть одна покупка из списка рекомендаций человеком, которому было предоставлено это предложение?
* Рекомендованы самые дорогие товары людям, которые покупают товары из дешевого ценового сегмента. Была ли хоть одна покупка в рамках этой рекомендации?

### 2. В метрике NDCG@k мы используем логарифм в знаменателе.<br>Как Вы думаете, почему именно логарифм?<br>Какую функцию можно использовать вместо логарифма?<br>Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в знаменателе присутствует логарифм.

В метрике NDCG@k используется логарифм по причине более плавного падения весов значений.  

Вместо логарифма можно попробовать использовать loss.

### 3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована).

In [5]:
from scipy.stats import spearmanr # коэффициенты корреляции Спирмена
from scipy.stats import kendalltau # коэффициенты корреляции Кенделла

In [36]:
a = [1, 2, 3, 4]
b = [1, 2, 3, 4]

In [37]:
kendalltau(a, b)

KendalltauResult(correlation=1.0, pvalue=0.08333333333333333)

In [38]:
spearmanr(a, b)

SpearmanrResult(correlation=1.0, pvalue=0.0)

### 4. Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8 

In [8]:
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]
boughted = [1, 3, 5, 7, 9, 11]

**pr@8**

In [9]:
pr_at_8 = (0 + 1 + 1 + 0 + 1 + 1 + 0 + 0) / 8
pr_at_8

0.5

In [10]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [11]:
precision_at_k(recommended, boughted, k=2)

0.5

**rec@8**

In [12]:
rec_at_8 = (0 + 1 + 1 + 0 + 1 + 1 + 0 + 0) / 6
rec_at_8

0.6666666666666666

In [13]:
def recall_at_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [14]:
recall_at_k(recommended, boughted)

0.6666666666666666

**AP@8**

In [15]:
AP_at_8 = (0 + 1/2 + 2/3 + 0 + 3/5 + 4/6 + 0 + 0)/4
AP_at_8

0.6083333333333333

In [16]:
def ap_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(recommended_list[:k], bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k

    result = sum_ / sum(flags)
    
    return result

In [17]:
ap_k(recommended, boughted, k=8)

0.6083333333333333

**NDCG@8**

In [18]:
DCG_at_8 = (0/1 + 1/np.log2(2) + 1/np.log2(3) + 0/np.log2(4) + 1/np.log2(5) + 1/np.log2(6) \
            + 0/np.log2(7) + 0/np.log2(8))/8
DCG_at_8

0.30605738985992403

In [19]:
ideal_DCG_at_8 = (1/1 + 1/np.log2(2) + 1/np.log2(3) + 1/np.log2(4) + 1/np.log2(5) + 1/np.log2(6) \
            + 1/np.log2(7) + 1/np.log2(8))/8
ideal_DCG_at_8

0.5797499549150934

In [20]:
NDCG_at_8 = DCG_at_8 / ideal_DCG_at_8
NDCG_at_8

0.5279127445637272

In [21]:
def dcg_at_k(recommended_list, bought_list, k=3):
    
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0: 
        return 0
    
    sum_ = 0
    for i in range(1, k):
        if i == 1 and flags[i-1] == True:
            sum_ += 1
        elif i != 1 and flags[i-1] == True:
            sum_ += 1/np.log2(i)
            
    return sum_ / k

In [22]:
dcg_at_k(recommended, boughted, k=8)

0.30605738985992403

In [23]:
def ideal_dcg_at_k(k=5):
    
    if k == 1: 
        return 1
    
    sum_ = 0
    for i in range(2, k+1):
        sum_ += 1/np.log2(i)
        
    return (1 + sum_)/k   

In [24]:
ideal_dcg_at_k(k=8)

0.5797499549150935

In [25]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    
    dcg = dcg_at_k(recommended_list, bought_list, k)
    ideal_dcg = ideal_dcg_at_k(k)
    
    return dcg / ideal_dcg

In [26]:
ndcg_at_k(recommended, boughted, k=8)

0.527912744563727

**RR@8**

In [27]:
rr_at_8 = 0/(1) + 1/(2)
rr_at_8

0.5

In [28]:
def reciprocal_rank_at_k(recommended_list, bought_list, k=5):
    
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    
    flags = np.isin(recommended_list, bought_list)

    for i in range(0, k):
        if flags[i]:
            ranks = 1 / (i+1)
            break
                
    return ranks

In [29]:
reciprocal_rank_at_k(recommended, boughted, k=8)

0.5

**ERR@8**

In [30]:
recommended

[2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

In [31]:
boughted

[1, 3, 5, 7, 9, 11]

In [32]:
err_at_8 = (0/(1+1) + 1/(2+1) + 1/(3+1) + 0/(4+1) + 1/(5+1) + 1/(6+1) + 0/(7+1) + 0/(8+1))/8
err_at_8

0.11160714285714285

In [33]:
def err_at_k(recommended_list, bought_list, k=5):
    
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    ranks = 0
    for i in range(0, k):
        if flags[i]:
            ranks += 1 / (i+2)
                
    return ranks / len(recommended_list)

In [34]:
err_at_k(recommended, boughted, k=8)

0.11160714285714285